In [1]:
import numpy as np
import requests
from pandas.io.json import json_normalize 
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import geocoder

In [2]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
location = get_latlng('M4G')

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0b515533562a4dbf9b381ac9c0564697 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='3Sh9U4ZAHhH1K6QSdPdIF1euvFPcbm7tnBXndxoYRgKc',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_0b515533562a4dbf9b381ac9c0564697.get_object(Bucket='capstoneproject-donotdelete-pr-h2zudmbgjtidbz',Key='work1a22.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

TorontoData = pd.read_csv(body)
TorontoData.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoData ['Borough'].unique()),
        TorontoData .shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [5]:
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = TorontoData['Latitude']
Y = TorontoData['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
TorontoData['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(TorontoData['Latitude'], TorontoData['Longitude'], TorontoData['Borough'], TorontoData['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map